# Chapter 4 Training Models

So far we have treated Machine Learning models and their training algorithms mostly like black boxes.If you went through some of the exercises in the previous chapters,you may have been surprised by how much you can get done without knowing anything about what's under the hood:you optimised a regression system,you improved a digit image classifer,and you even built a sapm classifer from scratch,all this without knowing how they actually work.Indeed, in many situations you do not really know the implementation details.